In [1]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops 

In [3]:
import os
import time
import numpy as np
import pandas as pd
from typing import Dict, List

import matplotlib.pyplot as plt

import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoTokenizer, 
    TrainingArguments, 
    Trainer,
    EarlyStoppingCallback
)

import warnings
warnings.filterwarnings("ignore")

d:\chatbot\envs\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.environ['WANDB_DISABLED']="true"

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
print(device)

cpu


### Load model

In [7]:
model_name = 'Biscottezi/vit5-base-finetuned-vitext2sql'
tokenizer = AutoTokenizer.from_pretrained(model_name)
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_model = original_model.to(device)

In [8]:
original_model

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

### Load dataset

In [9]:
dataset_train = load_dataset("huyhoangt2201/contextawareJidouka_fixed", split='train[:90%]')
dataset_val = load_dataset("huyhoangt2201/contextawareJidouka_fixed", split='train[-10%:]')
dataset = DatasetDict({
    'train': dataset_train,
    'validation': dataset_val
})
dataset.save_to_disk("completed_train_dataset")
print("Created train dataset!")

Saving the dataset (1/1 shards): 100%|██████████| 98/98 [00:00<00:00, 6998.48 examples/s]

Created train dataset!


### Preprocessing data

In [10]:

system_prompt = """You are an SQL query assistant. Based on schema, generate an SQL query to retrieve the relevant information for the user. If the user’s question is unrelated to the table, respond naturally in user's language.

Schema:
+Table Author, columns=[AuthorId: int, AuthorName: nvarchar(255), DepartmentId int, GroupDCId int]
+Table Department, columns=[DepartmentId: int, DepartmentName: nvarchar(255)]
+Table GroupDC, columns=[GroupDCId: int, DepartmentId: int, GroupDCName nvarchar(255)]
+Table Job, columns=[JobId: int, JobName: nvarchar(255)]
+Table Tool, columns=[ToolId: int, ToolName: nvarchar(255), ToolDescription: text]
+Table Jidouka, columns=[JidoukaId: bigint, ProductApply: nvarchar(255), ImprovementName: nvarchar(255), SoftwareUsing: nvarchar(255), Description: nvarchar(255), Video: text, DetailDocument: text, TotalJobApplied: int, TotalTimeSaved: int, DateCreate: datetime, JobId: int, AuthorId: int, DepartmentId: int, GroupDCId: int]
+Table JidoukaTool, columns=[JidoukaId: bigint, ToolId: int]
+Primary_keys=[Author.AuthorId, Department.DepartmentId, GroupDC.GroupDCId, Job.JobId, Tool.ToolId, Jidouka.JidoukaId]
+Foreign_keys=[GroupDC.DepartmentId=Department.DepartmentId, Jidouka.JobId=Job.JobId, Jidouka.AuthorId=Author.AuthorId, Jidouka.DepartmentId=Department.DepartmentId, Jidouka.GroupDCId=GroupDC.GroupDCId, JidoukaTool.JidoukaId=Jidouka.JidoukaId, JidoukaTool.ToolId=Tool.ToolId, Author.DepartmentId=Department.DepartmentId, Author.GroupDCId=GroupDC.GroupDCId]
"""

In [11]:
def format_context(sample):
    sample['context'] = system_prompt

    return sample

In [12]:
dataset_train2 = dataset_train.map(format_context)
dataset_val2 = dataset_val.map(format_context)

In [13]:
dataset_train3 = dataset_train2.shuffle(seed=42)
dataset_val3 = dataset_val2.shuffle(seed=42)

In [14]:
def tokenize_function(sample):
    """
    Convert dataset to instructions for LLM
    Args:
    sample: a record from dataset include id, context, questions, sql_answer
    """
    start_prompt = "Context:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:\n"

    data_zip = zip(sample['context'], sample['previous_question'])
    prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip]
    sample['input_ids'] = tokenizer(prompt, padding=True, truncation=True, return_tensors="pt").input_ids
    sample['labels'] = tokenizer(sample['previous_answer'], padding=True, truncation=True, return_tensors="pt").input_ids


    return sample

In [15]:
dataset_train3

Dataset({
    features: ['previous_question', 'previous_answer', 'schema_linking', 'question', 'answer', 'context'],
    num_rows: 879
})

In [16]:
tokenized_datasets_train = dataset_train3.map(tokenize_function, batched=True)
tokenized_datasets_train = tokenized_datasets_train.remove_columns(['previous_question', 'previous_answer', 'schema_linking', 'question', 'answer', 'context'])

In [17]:
tokenized_datasets_test = dataset_val3.map(tokenize_function, batched=True)
tokenized_datasets_test = tokenized_datasets_test.remove_columns(['previous_question', 'previous_answer', 'schema_linking', 'question', 'answer', 'context'])

### Test model before fine-tuning

In [17]:
question = dataset['validation'][0]['question']
context = dataset['validation'][0]['context']
answer = dataset['validation'][0]['answer']

prompt = f"""Context:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to(device)

output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

KeyError: 'context'

In [18]:
to_train = True
model_name = 'Biscottezi/vit5-base-finetuned-vitext2sql'
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
finetuned_model = finetuned_model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [23]:
# finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_10_epoch")
# finetuned_model = finetuned_model.to(device)

In [19]:
import accelerate

In [20]:
%%time

if to_train:
    output_dir = f'sql-training-{str(int(time.time()))}'

    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=5e-3,
        num_train_epochs=100,
        per_device_train_batch_size=16,     # batch size per device during training
        per_device_eval_batch_size=16,      # batch size for evaluation
        weight_decay=0.01,
        logging_steps=50,
        eval_strategy='epoch',
        save_strategy='epoch',        # evaluation strategy to adopt during training
        eval_steps=50,        
        load_best_model_at_end=True             # number of steps between evaluation
    )

    early_stopping_callback = EarlyStoppingCallback( 
        early_stopping_patience=5
    )

    trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        train_dataset=tokenized_datasets_train,
        eval_dataset=tokenized_datasets_test,
        callbacks=[early_stopping_callback]
    )
    
    trainer.train()
    
    finetuned_model.save_pretrained("finetuned_model_100_epoch")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
  0%|          | 0/5500 [00:00<?, ?it/s]

: 

In [16]:
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_100_epoch")
finetuned_model = finetuned_model.to(device)

In [18]:
question = "Công cụ nào giúp tôi tiết kiệm thời gian khi làm việc?"
context = """Có 1 bảng cần truy vấn. 
Bảng cần truy vấn bao gồm các cột: 
id: số thứ tự của hàng (int);
innovation_name: tên của tác phẩm cải tiến (str);
task_type: Tác phẩm cải tiến đó sinh ra để làm gì? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) ;
tool: Công cụ để thực hiện (str) (ví dụ: Python, Excel, Visual Studio Code, ...);
describe_innovation: Mô tả rõ ràng hơn mục đích của công cụ (giải thích rõ hơn cột task_type) (str)  ;
product: Output của công cụ có định dạng như thế nào (str) (ví dụ: file csv, file xlsx, ....);
pic: Tên người phụ trách quản lí công cụ (str)  ;
dc: Phòng ban làm việc của người phụ trách quản lí công cụ (str) (dc1, dc2, dc3, dcd, souko,...);
saved_hours: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);
created_at: Thời điểm công cụ này ra mắt (str) (ví dụ: 2024-10-11, 2024-10-10,...);
information: Đường link youtube tài liệu hướng dẫn sử dụng công cụ (str)"""
answer = """
SELECT innovation_name, saved_hours FROM jidouka ORDER BY saved_hours DESC LIMIT 1;"""
prompt = f"""Context:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to(device)

output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'FINE-TUNED MODEL - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Context:
Có 1 bảng cần truy vấn. 
Bảng cần truy vấn bao gồm các cột: 
id: số thứ tự của hàng (int);
innovation_name: tên của tác phẩm cải tiến (str);
task_type: Tác phẩm cải tiến đó sinh ra để làm gì? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) ;
tool: Công cụ để thực hiện (str) (ví dụ: Python, Excel, Visual Studio Code, ...);
describe_innovation: Mô tả rõ ràng hơn mục đích của công cụ (giải thích rõ hơn cột task_type) (str)  ;
product: Output của công cụ có định dạng như thế nào (str) (ví dụ: file csv, file xlsx, ....);
pic: Tên người phụ trách quản lí công cụ (str)  ;
dc: Phòng ban làm việc của người phụ trách quản lí công cụ (str) (dc1, dc2, dc3, dcd, souko,...);
saved_hours: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);
created_at: Thời điểm công cụ này ra mắt (str) (ví dụ: 2024-10-11, 2024-10-10,...);
information: Đườn